In [5]:
import numpy as np
import pandas as pd
import os
import json
import openai
import sys
import importlib
import json_repair as jr
from dotenv import load_dotenv
import anthropic
import random
from google import genai

import creators_sct as cr
import judges_sct as jg


In [85]:
importlib.reload(jg)
importlib.reload(cr)

<module 'creators_sct' from '/home/isaac/Documents/VUMC/reasyn25/creator/creators_sct.py'>

In [4]:
import os
import sys
cwd = os.getcwd()
judge_path = os.path.join(cwd, "judge")
creator_path = os.path.join(cwd, "creator")

sys.path.append(judge_path)
sys.path.append(creator_path)

In [121]:
input = ["psychiatry", True, True]
goob = cr.co41_creator_synth_sct(input)

In [122]:
goob

array(['A 28-year-old female graduate student presents with a 3-month history of excessive worry about multiple aspects of her life, including academic performance and relationships. She reports difficulty concentrating, muscle tension, and insomnia, sleeping only 3-4 hours nightly. She denies substance use and has no significant medical history. Mental status exam reveals an anxious affect, rapid speech, and hypervigilance to environmental stimuli.',
       'Generalized anxiety disorder; consider starting SSRI',
       'Upon further questioning, she reveals consuming 6-8 cups of coffee daily and 2-3 energy drinks to maintain her productivity, a pattern that started 4 months ago when she began her thesis',
       "This subsequent finding creates complexity because it introduces caffeine-induced anxiety disorder as a differential diagnosis that could fully explain the patient's symptoms. The clinician must now consider whether the symptoms are primarily due to GAD, caffeine intoxication

In [128]:
jginput = [input[0], 'co41']
jgtest = jg.co41_judge_api_sct('psychiatry', goob)

#jgtest = jg.sct_judge_synth(jginput, goob, 3)
#udge1_temp = jg.sct_judge_synth(jgtest, 'psychiatry', goob, 3)



In [129]:
jgtest

'```json\n{\n  "realistic": 5,\n  "medically_accurate": 5,\n  "diverse_demographics": 3\n}\n```'

In [ ]:
import random
import numpy as np
#monitor API status pages closely when running large batches, these methods work even when some APIs do not and can lead to wasted expenditure
def get_creators():
    # Returns a dictionary of creator functions.
    return {
        "gpt5": cr.gpt5_creator_synth_sct,
        "gpt41": cr.gpt41_creator_synth_sct,
        "ds": cr.ds_creator_synth_sct,
        "cs45": cr.cs45_creator_synth_sct,
        "o3": cr.o3_creator_synth_sct,
        "gem25p": cr.gem25p_creator_synth_sct,
        "co41": cr.co41_creator_synth_sct,
        "k2": cr.k2_creator_synth_sct
    }

def get_judges(): 
    # Returns a dictionary of judge functions.
    return {
        "gpt5",
        "gpt41",
        "ds",
        "cs45",
        "o3",
        "gem25p",
        "co41",
        "k2"
    }

def pick_creator_judge(creators, judges):
    """
    Randomly selects two *different* models: one for creator, one for judge.
    Returns (creator_name, creator_func, judge_name, judge_func)
    """
    # Pick a random creator
    creator_name = random.choice(list(creators.keys()))
    
    # Pick a judge from the remaining options
    remaining_judges = [name for name in judges if name != creator_name]
    #remaining_judges = [name for name in judges.keys()]
    judge1_name = random.choice(remaining_judges)
    #print(len(remaining_judges))
    remaining_judges = [name for name in judges if (name != creator_name and name != judge1_name)]
    judge2_name = random.choice(remaining_judges)
    #print(len(remaining_judges))
    return creator_name, creators[creator_name], judge1_name, judge2_name

#input data should be [domain (str), shot(bool), explanation ()]
def batch_sct(n, input_data):
    creators = get_creators()
    judges = get_judges()
    domain = input_data[0]
    arr = np.empty((0, 16))  # You can adjust this based on your data shape

    for i in range(n):
        creator_name, creator_func, judge1_name, judge2_name = pick_creator_judge(creators, judges)
        # Creator step
        temp = creator_func(input_data)
        # Judge step
        if temp[0][0] == "Error":
            #TODO change error length
            judge1_temp = np.array([(0, 0, 0, "Error")])
            judge2_temp = np.array([(0, 0, 0, "Error")])
        else:

            judge1_temp = jg.sct_judge_synth(judge1_name, domain, temp, 3)
            judge2_temp = jg.sct_judge_synth(judge2_name, domain, temp, 3)

        judge1_func = ""
        # Prepare data for concatenation
        temp = np.atleast_2d(temp)
        judge1_temp = np.atleast_2d(judge1_temp)
        judge2_temp = np.atleast_2d(judge2_temp)
        input_2d = np.atleast_2d(input_data)

        # Combine everything into one row
        combined = np.concatenate((input_2d, temp, judge1_temp, judge2_temp), axis=1)
        arr = np.vstack((arr, combined))
    return arr


In [95]:
input = ['cardiovascular', False, False]
goob3 = batch_sct(30, input)
goob3_pd = pd.DataFrame(goob3)
goob3_pd.to_csv("data/sct_trial104.csv")



In [53]:
goob2_pd = pd.DataFrame(goob2)
goob2_pd.to_csv("data/sct_trial20.csv")

In [96]:
goob3_pd

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,cardiovascular,False,False,A 68-year-old male presents to the emergency d...,"Acute coronary syndrome; obtain ECG, cardiac b...",The patient mentions that the pain significant...,,Opus 4.1,5,5,3,K2 Thinking,5,5,3,DeepSeek
1,cardiovascular,False,False,A 62-year-old male with a 25-year history of p...,Order stat chest CT angiogram to rule out aort...,"Shortly after arrival, the patient develops ne...",,GPT 4.1,5,5,3,o3,5,5,3,Claude Opus 4.1
2,cardiovascular,False,False,A 58-year-old male with a history of hypertens...,Begin evaluation for non-ST elevation myocardi...,"Shortly after arrival, the patient becomes acu...",,GPT 4.1,5,5,3,K2 Thinking,5,5,2,o3
3,cardiovascular,False,False,A 58-year-old male presents to the emergency d...,Acute inferior wall ST-elevation myocardial in...,The patient suddenly develops hypotension with...,,Sonnet 4.5,5,4,2,o3,5,5,2,GPT 5
4,cardiovascular,False,False,"A 62-year-old female with hypertension, hyperl...","Posterior STEMI. Activate cath lab, load dual ...",The patient's husband mentions she has been ta...,,K2 Thinking,4,5,3,GPT 5,5,5,3,Gemini 2.5 Pro
5,cardiovascular,False,False,A 62-year-old man with a history of poorly con...,"Possible NSTEMI; start aspirin, heparin, beta-...",Forty minutes later the patient reports sudden...,,o3,5,5,3,GPT 4.1,5,5,3,Claude Sonnet 4.5
6,cardiovascular,False,False,A 58-year-old male with a history of hypertens...,Acute inferior ST-elevation myocardial infarct...,"While preparing for catheterization, the patie...",,Sonnet 4.5,5,5,3,DeepSeek,5,5,2,o3
7,cardiovascular,False,False,"68-year-old male with hypertension, hyperlipid...","NSTEMI - initiate dual antiplatelet therapy, a...",CT angiography of the chest obtained to evalua...,,K2 Thinking,4,4,2,GPT 5,5,5,2,GPT 4.1
8,cardiovascular,False,False,A 68-year-old male with a history of hypertens...,Acute decompensated heart failure with preserv...,A transthoracic echocardiogram reveals a prese...,,Gemini 2.5 Pro,5,5,3,Claude Sonnet 4.5,5,5,2,o3
9,cardiovascular,False,False,A 68-year-old male presents to the emergency d...,"NSTEMI - initiate dual antiplatelet therapy, a...",The patient mentions he has been experiencing ...,,Opus 4.1,5,5,3,DeepSeek,5,4,2,o3


In [ ]:
load_dotenv()
api_key = os.getenv("DEEPSEEK_API_KEY")
client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")
prompt = cr.create_sct_prompt()
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a medical expert."},
        {"role": "user", "content": prompt},
    ],
    stream=False
)
return response.choices[0].message.content


In [92]:
input = ['cardiovascular', False, True]
goob = batch_sct(1, input)

In [93]:
print(goob)

[['cardiovascular' 'False' 'True'
  "A 68-year-old male with a history of well-controlled hypertension and hyperlipidemia presents to the clinic with a two-week history of worsening shortness of breath on exertion. He denies typical anginal chest pain but describes a 'pressure' in his upper chest when walking up a flight of stairs, which resolves with rest. His initial ECG shows non-specific T-wave flattening in the lateral leads, and his initial high-sensitivity troponin from the emergency department yesterday was 12 ng/L (normal <14 ng/L)."
  'Rule out non-ST-elevation acute coronary syndrome (NSTE-ACS); consider stress testing for stable coronary artery disease.'
  'On a more detailed physical examination, a 3/6 late-peaking systolic ejection murmur is auscultated at the right upper sternal border with radiation to the carotid arteries.'
  'This subsequent finding introduces a compelling alternative diagnosis—aortic stenosis—which perfectly mimics the symptoms of coronary artery dis